## Prompts for locations

In [1]:
location_prompts = [
    "steampunk city with skyscrapers",
    "cyberpunk village in Japanese rustic style",
    "fantasy dungeons and dragons",
    "noir city from 1930s",
    "StarTrek inspired spaceship",
    "undeground mine of goblins",
    "SuperMario style magic land plain",
    "SuperMario style magic land beach",
]

## Generating locations 

In [2]:
from holodeck.game_objects import initialize_location
from holodeck.gpt_text import generate_location_and_encounters

from tqdm.notebook import tqdm

locations = []

for p in tqdm(location_prompts, desc="Generating locations"):
    location_dict, encounters_list = generate_location_and_encounters(p)
    if location_dict:
        location = initialize_location(location_dict, encounters_list)
        locations.append(location)


locations

Generating locations:   0%|          | 0/8 [00:00<?, ?it/s]

ParserError: while parsing a block collection
  in "<unicode string>", line 8, column 3:
      - name: City Tower
      ^
expected <block end>, but found '<block sequence start>'
  in "<unicode string>", line 11, column 4:
       - name: Magenta Hub
       ^

## Generate Image Prompts

In [ ]:
from holodeck.gpt_text import \
        generate_object_image_prompt, \
        generate_building_image_prompt, \
        generate_location_image_prompt

locations_prompts = []
objects_image_prompts = []
buildings_image_prompts = []

for location in tqdm(locations, desc="Locations"):

    locations_prompts.append((location, generate_location_image_prompt(location)))

    for o in tqdm(location.objects, desc="Objects"):
        objects_image_prompts.append((o, generate_object_image_prompt(o, location)))


    for b in tqdm(location.objects, desc="Buildings"):
        objects_image_prompts.append((b, generate_building_image_prompt(o, location)))




Locations:   0%|          | 0/8 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'description'

## Generate Images

In [ ]:
from holodeck.gpt_image import generate_image
from IPython.display import display, Markdown

img_prompts = locations_prompts + objects_image_prompts + buildings_image_prompts

images = []

for obj, prompt in tqdm(img_prompts, desc="Images"):
    image = generate_image(prompt)
    display(Markdown(prompt))
    display(image)
    image_file_name = f".images/{obj.name}.jpg"
    image.save(image_file_name)
    images.append((obj, prompt, image_file_name))

len(images)


## Display Images

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import PIL.Image as Image

# create an array of tuples containing objects, text, and images
data = [
    ({"name": "Object 1"}, "Text 1", "image1.jpg"),
    ({"name": "Object 2"}, "Text 2", "image2.jpg"),
    ({"name": "Object 3"}, "Text 3", "image3.jpg"),
    ({"name": "Object 4"}, "Text 4", "image4.jpg"),
    ({"name": "Object 5"}, "Text 5", "image5.jpg"),
    ({"name": "Object 6"}, "Text 6", "image6.jpg"),
    ({"name": "Object 7"}, "Text 7", "image7.jpg"),
    ({"name": "Object 8"}, "Text 8", "image8.jpg")
]

# define the grid layout
grid = widgets.GridBox(
    layout=widgets.Layout(
        grid_template_columns="repeat(4, 1fr)",
        grid_gap="10px"
    )
)

# loop through the data and create a widget for each tuple
for i, (obj, text, img_path) in enumerate(data):
    with open(img_path, "rb") as f:
        img = Image.open(f)
        img_widget = widgets.Image(value=f.read(), format="jpg", width=150, height=150)
    obj_widget = widgets.HTML(f"<h3>{obj['name']}</h3>")
    text_widget = widgets.HTML(f"<p>{text}</p>")
    item = widgets.VBox([img_widget, obj_widget, text_widget])
    grid.children += (item,)
    
# display the grid
display(grid)


FileNotFoundError: [Errno 2] No such file or directory: 'image1.jpg'